In [ ]:
!pip install tensorflow
!pip install matplotlib
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.1 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import io
import ipywidgets as widgets


# Introduction
This lab is designed to introduce you to the basics of deep learning by interacting with a pre-built model. You'll understand the workflow of a deep learning project, including data preprocessing, model architecture, and making predictions. The goal is to familiarize yourself with the basics of deep learning without writing any code.


In [ ]:
# Load the VGG16 model
model = VGG16(weights='imagenet')

# Display the model architecture
model.summary()


553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 1000)           │     4,097,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,357,544 (527.79 MB)

 Trainable params: 138,357,544 (527.79 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Load and preprocess an image
def load_and_preprocess_image(image_path):
    # Load the image
    img = load_img(image_path, target_size=(224, 224))

    # Convert the image to a numpy array
    img_array = img_to_array(img)

    # Expand dimensions to fit the model input
    img_array = np.expand_dims(img_array, axis=0)

    # Preprocess the image
    img_array = preprocess_input(img_array)

    return img, img_array

# Load and preprocess a sample image
sample_image, processed_image = load_and_preprocess_image('sample.jpg')

# Display the sample image
plt.imshow(sample_image)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'sample.jpg'

In [ ]:
# Make predictions
# This cell will now process the image data from the upload widget
def make_prediction(img_data):
    img = Image.open(io.BytesIO(img_data))
    img = img.resize((224, 224))

    # Preprocess and predict
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    predictions = model.predict(img_array)
    decoded_predictions = decode_predictions(predictions, top=3)[0]

    return decoded_predictions

# Note: This cell now defines the prediction function, but does not execute it directly.
# The prediction will be triggered by the button click in the next cell.

In [ ]:
# Upload button to load images
upload = widgets.FileUpload()
display(upload)

# Button to make predictions
predict_button = widgets.Button(description="Make Prediction")
display(predict_button)

# Function to handle button click
def on_click(change):
    img_data = list(upload.value.values())[0]['content']
    decoded_predictions = make_prediction(img_data) # Use the function from the previous cell

    # Display predictions
    print(decoded_predictions)

predict_button.on_click(on_click)

FileUpload(value={}, description='Upload')

Button(description='Make Prediction', style=ButtonStyle())

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
[('n02123394', 'Persian_cat', np.float32(0.8026809)), ('n04493381', 'tub', np.float32(0.020179704)), ('n02488291', 'langur', np.float32(0.0142714055))]


# Conclusion and Discussion
Reflect on the lab activities. Discuss how the pre-trained model was able to make predictions, the role of data preprocessing, and the impact of input modifications on the model's predictions.


# Task
Complete the notebook by adding interactive widgets to modify images and observe the effect on model predictions, and guide the user through the remaining steps of the lab.

## Add image modification widgets

### Subtask:
Generate code cells to create interactive widgets (e.g., sliders, buttons) for rotating, adding noise, or other suitable image transformations.


**Reasoning**:
Create interactive widgets for image transformations and define a function to apply these transformations.



In [ ]:
# Create widgets for transformations
rotation_slider = widgets.IntSlider(
    value=0,
    min=-180,
    max=180,
    step=1,
    description='Rotation Angle:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

noise_slider = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=0.5,
    step=0.01,
    description='Noise Intensity:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)

# Define function to apply transformations
def apply_transformations(img_data, rotation_angle, noise_intensity):
    img = Image.open(io.BytesIO(img_data))

    # Apply rotation
    rotated_img = img.rotate(rotation_angle)

    # Apply noise
    if noise_intensity > 0:
        img_array = np.array(rotated_img)
        noise = np.random.normal(0, noise_intensity * 255, img_array.shape).astype('uint8')
        noisy_img_array = np.clip(img_array + noise, 0, 255)
        transformed_img = Image.fromarray(noisy_img_array)
    else:
        transformed_img = rotated_img

    return transformed_img

# Display the widgets
display(rotation_slider)
display(noise_slider)

IntSlider(value=0, continuous_update=False, description='Rotation Angle:', max=180, min=-180)

FloatSlider(value=0.0, continuous_update=False, description='Noise Intensity:', max=0.5, step=0.01)

**Reasoning**:
Connect the widgets to the image transformation and prediction function and display the transformed image and predictions.



In [ ]:
# Create an output widget to display the image and predictions
output_widget = widgets.Output()
display(output_widget)

# Function to update the output based on widget changes
def update_output(change):
    with output_widget:
        output_widget.clear_output()
        if upload.value:
            img_data = list(upload.value.values())[0]['content']
            rotation_angle = rotation_slider.value
            noise_intensity = noise_slider.value

            # Apply transformations
            transformed_img = apply_transformations(img_data, rotation_angle, noise_intensity)

            # Display transformed image
            plt.imshow(transformed_img)
            plt.title("Transformed Image")
            plt.axis('off')
            plt.show()

            # Make and display predictions
            decoded_predictions = make_prediction(transformed_img.tobytes()) # Pass bytes data to make_prediction
            print("Predictions:", decoded_predictions)


# Observe changes in the widgets and trigger the update function
rotation_slider.observe(update_output, names='value')
noise_slider.observe(update_output, names='value')
upload.observe(update_output, names='value') # Also update when a new image is uploaded

Output()